# Settings

## Constants

In [1]:
import os

PATH_DATA = './Dataset'
sensor_data = ['UserInfo.csv', 'Service.csv', 'ContextualFactor.csv', 'Interruptibility.csv']

# Utility Functions

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneGroupOut
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score


# Load the Dataset into Dataframe

In [3]:
dataframes = {
    filename: pd.read_csv(os.path.join(PATH_DATA, filename)).reset_index(drop=True)
    for filename in sensor_data
}
dfService = dataframes['Service.csv']
dfContextualFactor = dataframes['ContextualFactor.csv']
dfUserInfo = dataframes['UserInfo.csv']
dfInterruptibility = dataframes['Interruptibility.csv']

# Preprocessing

In [4]:
# Select specific columns from dfContextualFactor, dfService, and dfUserInfo DataFrames
# Combine relevant columns to create a unified dataset
dfContextualFactor_selected_columns=dfContextualFactor[['uid','sid','activity1','activity2','activity3','userLocation','userPosition',]]
dfService_selected_columns=dfService[['weekOfExperiment','dayOfWeek','startTime', 'activityInquiry','availabilityInquiry','speechShadowing_1','speechShadowing_2','speechShadowing_3','speechShadowing_4','speechShadowing_5','continue-to-nextInquiry_1','continue-to-nextInquiry_2','continue-to-nextInquiry_3','continue-to-nextInquiry_4','endTime','endType']]
dfInterruptibility_selected_columns=dfInterruptibility[['SHORT_INTERACTION_interruptibility', 'LONG_INTERACTION_interruptibility']]

## Create binary columns for interaction types
## SHORT_INTERACTION: True if availabilityInquiry is not NaN
# dfCombinedAll['SHORT_INTERACTION_interruptibility'] = dfCombinedAll['availabilityInquiry'].notna()
## LONG_INTERACTION: True if continue-to-nextInquiry_1 is not NaN
# dfCombinedAll['LONG_INTERACTION_interruptibility'] = dfCombinedAll['continue-to-nextInquiry_1'].notna()

# ! For those who want to redefine LONG_INTERACTION with thresholds longer than 3 minutes,
# you can use the following columns:
# 5 minutes  => use continue-to-nextInquiry_2
# 7 minutes  => use continue-to-nextInquiry_3
# 9 minutes  => use continue-to-nextInquiry_4


dfCombinedAll=pd.concat([dfContextualFactor_selected_columns, dfService_selected_columns,dfInterruptibility_selected_columns], axis=1)

In [5]:
# Concatenate and merge the all columns from dfUserInfo (dfUserInfo.csv)
dfUserInfo_selected_columns = dfUserInfo[['uid', 'homeType', 'speakerLocation', 'speakerPosition']]
dfCombinedAll = pd.merge(dfCombinedAll, dfUserInfo_selected_columns, on='uid', how='left')

## Position Processing

In [6]:
# Define function to calculate proximity between user and speaker
def calculate_proximity(row):
    # Return 0 if user and speaker are in different locations
    if row['userLocation'] != row['speakerLocation']:
        return 0
    # If in the same locations
    elif row['userPosition'] == row['speakerPosition']:
        return 2  # Same position
    else:
        return 1  # Different positions (including missing position)

# Apply proximity calculation to create a new 'proximity' column
dfCombinedAll['proximity'] = dfCombinedAll.apply(calculate_proximity, axis=1)

print(dfCombinedAll[['userLocation', 'userPosition', 'speakerLocation', 'speakerPosition', 'proximity']].head(10))


  userLocation userPosition speakerLocation speakerPosition  proximity
0     Bed Room          Bed        Bed Room            Desk          1
1    Rest Room          NaN        Bed Room            Desk          0
2  Living Room          NaN        Bed Room            Desk          0
3  Living Room          NaN        Bed Room            Desk          0
4     Bed Room          Bed        Bed Room            Desk          1
5     Bed Room          Bed        Bed Room            Desk          1
6     Bed Room          Bed        Bed Room            Desk          1
7     Bed Room         Desk        Bed Room            Desk          2
8     Bed Room          Bed        Bed Room            Desk          1
9     Bed Room          Bed        Bed Room            Desk          1


## Activity and Time Processing

In [7]:
# Process activity columns for one-hot encoding
activity_cols = ['activity1', 'activity2', 'activity3']
df_activity = dfContextualFactor[activity_cols].copy()

# Get unique activities across all activity columns, excluding NaN
all_unique_activities = pd.unique(df_activity.values.ravel())
all_unique_activities = [x for x in all_unique_activities if pd.notna(x)]

# Create a DataFrame for one-hot encoding of activities
dfActivity_one_hot_encoding = pd.DataFrame(0, index=df_activity.index, columns=['act_' + str(val) for val in all_unique_activities])

# Perform one-hot encoding for each activity column
for col in activity_cols:
    for val in all_unique_activities:
        dfActivity_one_hot_encoding['act_' + str(val)] |= (df_activity[col] == val).astype(int)

# Concatenate one-hot encoded activity columns to dfCombinedAll
dfActivity_one_hot_encoding
dfCombinedAll = pd.concat([dfCombinedAll, dfActivity_one_hot_encoding], axis=1)

In [8]:
# Convert startTime to datetime and extract total minutes since midnight
dfCombinedAll['startTime'] = pd.to_datetime(dfCombinedAll['startTime'], format='%H:%M:%S', errors='coerce')
dfCombinedAll['minute'] = dfCombinedAll['startTime'].dt.hour * 60 + dfCombinedAll['startTime'].dt.minute

# Map days of the week to numerical values (MON=0, TUE=1, ..., SUN=6)
day_map = {'MON': 0, 'TUE': 1, 'WED': 2, 'THU': 3, 'FRI': 4, 'SAT': 5, 'SUN': 6}
dfCombinedAll['dayOfWeek'] = dfCombinedAll['dayOfWeek'].map(day_map)

# Bin minutes into 30-minute intervals for temporal analysis
dfCombinedAll['minute_bin'] = (dfCombinedAll['minute'] // 30).astype(int)

In [9]:
dfCombinedAll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830 entries, 0 to 2829
Data columns (total 43 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   uid                                       2830 non-null   int64         
 1   sid                                       2830 non-null   int64         
 2   activity1                                 2830 non-null   object        
 3   activity2                                 110 non-null    object        
 4   activity3                                 3 non-null      object        
 5   userLocation                              2830 non-null   object        
 6   userPosition                              2343 non-null   object        
 7   weekOfExperiment                          2830 non-null   int64         
 8   dayOfWeek                                 2830 non-null   int64         
 9   startTime                     

# Feature Extraction

In [15]:
# Select features for response prediction
dfFeatresForResponse = dfCombinedAll[['uid',
    'act_Taking a Nap / Sleeping','act_Hygiene','act_Eating','act_Using Media','act_Social Interaction',
    'act_Returning from Outside / Other Rooms','act_Studying / Working','act_Others','act_House Chores',
    'act_Self Caring','act_Visiting Outside / Other Rooms','act_Resting',
    'homeType','userLocation','userPosition','speakerLocation','speakerPosition',
    'minute_bin','dayOfWeek','SHORT_INTERACTION_interruptibility','LONG_INTERACTION_interruptibility']].copy()

# Encode categorical columns using LabelEncoder
categorical_columns = ['homeType', 'userLocation', 'userPosition', 'speakerLocation', 'speakerPosition', 'minute_bin','SHORT_INTERACTION_interruptibility','LONG_INTERACTION_interruptibility']
label_encoders = defaultdict(LabelEncoder)

# Apply label encoding to each categorical column
for col in categorical_columns:
    dfFeatresForResponse[col] = label_encoders[col].fit_transform(dfFeatresForResponse[col])

# Create a copy of the encoded data for further processing
encoded_data = dfFeatresForResponse.copy()


encoded_data.info()
encoded_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830 entries, 0 to 2829
Data columns (total 22 columns):
 #   Column                                    Non-Null Count  Dtype
---  ------                                    --------------  -----
 0   uid                                       2830 non-null   int64
 1   act_Taking a Nap / Sleeping               2830 non-null   int64
 2   act_Hygiene                               2830 non-null   int64
 3   act_Eating                                2830 non-null   int64
 4   act_Using Media                           2830 non-null   int64
 5   act_Social Interaction                    2830 non-null   int64
 6   act_Returning from Outside / Other Rooms  2830 non-null   int64
 7   act_Studying / Working                    2830 non-null   int64
 8   act_Others                                2830 non-null   int64
 9   act_House Chores                          2830 non-null   int64
 10  act_Self Caring                           2830 non-null   in

,uid,act_Taking a Nap / Sleeping,act_Hygiene,act_Eating,act_Using Media,act_Social Interaction,act_Returning from Outside / Other Rooms,act_Studying / Working,act_Others,act_House Chores,...,act_Resting,homeType,userLocation,userPosition,speakerLocation,speakerPosition,minute_bin,dayOfWeek,SHORT_INTERACTION_interruptibility,LONG_INTERACTION_interruptibility
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,12,0,0,1
1,1,0,1,0,0,0,0,0,0,0,...,0,0,3,3,0,1,15,0,1,1
2,1,0,0,1,0,0,0,0,0,0,...,0,0,1,3,0,1,12,1,1,1
3,1,0,0,1,0,0,0,0,0,0,...,0,0,1,3,0,1,13,1,1,1
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,18,1,0,1


## Label: SHORT_INTERACTION

In [16]:
# Label distribution before balancing
print(encoded_data['SHORT_INTERACTION_interruptibility'].value_counts())

SHORT_INTERACTION_interruptibility
0    2088
1     742
Name: count, dtype: int64


### Model Building and LOSO CV

In [17]:
# Prepare features (X) and target variables (y) for SHORT_INTERACTION
X = encoded_data.drop(columns=['SHORT_INTERACTION_interruptibility', 'LONG_INTERACTION_interruptibility', 'uid'], axis=1) # Drop target and unrelated columns
y = encoded_data['SHORT_INTERACTION_interruptibility'] # Target variable
groups = encoded_data['uid'] # Group by user ID for Leave-One-Group-Out CV

# Initialize Leave-One-Group-Out cross-validator
logo = LeaveOneGroupOut()

# Initialize SMOTE for oversampling to handle class imbalance
smote = SMOTE(random_state=42)

# Define models to evaluate
models1 = {
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'LightGBM': LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'CatBoost': CatBoostClassifier(iterations=100, depth=5, learning_rate=0.1, loss_function='Logloss', cat_features=[0], random_seed=42),
    'SVM': SVC(random_state=42),
    'Dummy': DummyClassifier(strategy="stratified", random_state=42)
}

results1_logo = {}

# Loop over each model
for model_name, model1 in models1.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in Leave-One-Group-Out cross-validation
    for i, (train_index, test_index) in enumerate(logo.split(X, y, groups)):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model1.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model1.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)
    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results1_logo[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }


C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecate

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2034, number of negative: 2034
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000544 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 87
[LightGBM] [Info] Number of data points in the train set: 4068, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecate

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecate

...

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6544117	total: 927us	remaining: 91.8ms
1:	learn: 0.6249201	total: 1.71ms	remaining: 84ms
2:	learn: 0.5963593	total: 2.55ms	remaining: 82.6ms
3:	learn: 0.5738709	total: 3.35ms	remaining: 80.3ms
4:	learn: 0.5533208	total: 4.14ms	remaining: 78.8ms
5:	learn: 0.5366001	total: 5.13ms	remaining: 80.4ms
6:	learn: 0.5178801	total: 6.03ms	remaining: 80.1ms
7:	learn: 0.5033031	total: 6.84ms	remaining: 78.7ms
8:	learn: 0.4884807	total: 7.8ms	remaining: 78.9ms
9:	learn: 0.4789881	total: 8.6ms	remaining: 77.4ms
10:	learn: 0.4685460	total: 9.41ms	remaining: 76.2ms
11:	learn: 0.4609102	total: 10.2ms	remaining: 74.8ms
12:	learn: 0.4515636	total: 11.1ms	remaining: 74.1ms
13:	learn: 0.4432493	total: 11.9ms	remaining: 73.1ms
14:	learn: 0.4384408	total: 12.8ms	remaining: 72.6ms
15:	learn: 0.4311124	total: 13.6ms	remaining: 71.6ms
16:	learn: 0.4266337	total: 14.6ms	remaining: 71.4ms
17:	learn: 0.4208247	total: 15.5ms	remaining: 70.5ms
18:	learn: 0.4161441	total: 16.4ms	remaining: 69.8ms
19:	lear

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


54:	learn: 0.3587207	total: 50.6ms	remaining: 41.4ms
55:	learn: 0.3579581	total: 51.6ms	remaining: 40.6ms
56:	learn: 0.3571185	total: 52.5ms	remaining: 39.6ms
57:	learn: 0.3566297	total: 53.4ms	remaining: 38.7ms
58:	learn: 0.3556686	total: 54.4ms	remaining: 37.8ms
59:	learn: 0.3548915	total: 55.2ms	remaining: 36.8ms
60:	learn: 0.3542693	total: 56.3ms	remaining: 36ms
61:	learn: 0.3535535	total: 57.4ms	remaining: 35.2ms
62:	learn: 0.3527940	total: 58.4ms	remaining: 34.3ms
63:	learn: 0.3521635	total: 59.6ms	remaining: 33.5ms
64:	learn: 0.3513379	total: 60.6ms	remaining: 32.6ms
65:	learn: 0.3496508	total: 61.6ms	remaining: 31.7ms
66:	learn: 0.3489191	total: 62.6ms	remaining: 30.8ms
67:	learn: 0.3479739	total: 63.5ms	remaining: 29.9ms
68:	learn: 0.3475503	total: 64.6ms	remaining: 29ms
69:	learn: 0.3469609	total: 65.6ms	remaining: 28.1ms
70:	learn: 0.3464168	total: 66.7ms	remaining: 27.2ms
71:	learn: 0.3454552	total: 67.8ms	remaining: 26.4ms
72:	learn: 0.3449885	total: 68.8ms	remaining: 25.5

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\368976191.py:40: FutureWarning: Series.ravel is deprecate

### K-Fold CV

In [18]:
from sklearn.model_selection import KFold

# Initialize 5-fold cross-validator
kfold = KFold(n_splits=5, shuffle=False)

results1_kfold = {}

# Loop over each model
for model_name, model1 in models1.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in 5-fold cross-validation
    for train_index, test_index in kfold.split(X, y):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model1.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model1.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)

    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results1_kfold[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }


C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\2766920084.py:23: FutureWarning: Series.ravel is depre

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1695, number of negative: 1695
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000196 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 87
[LightGBM] [Info] Number of data points in the train set: 3390, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\2766920084.py:23: FutureWarning: Series.ravel is depre

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6547612	total: 986us	remaining: 97.7ms
1:	learn: 0.6216123	total: 1.91ms	remaining: 93.6ms
2:	learn: 0.5940315	total: 2.68ms	remaining: 86.8ms
3:	learn: 0.5710338	total: 3.49ms	remaining: 83.7ms
4:	learn: 0.5533896	total: 4.25ms	remaining: 80.7ms
5:	learn: 0.5288291	total: 5.07ms	remaining: 79.5ms
6:	learn: 0.5085043	total: 5.88ms	remaining: 78.1ms
7:	learn: 0.4914158	total: 6.68ms	remaining: 76.8ms
8:	learn: 0.4817332	total: 7.28ms	remaining: 73.6ms
9:	learn: 0.4680142	total: 8.1ms	remaining: 72.9ms
10:	learn: 0.4567986	total: 8.91ms	remaining: 72.1ms
11:	learn: 0.4465935	total: 9.7ms	remaining: 71.1ms
12:	learn: 0.4386785	total: 10.7ms	remaining: 71.8ms
13:	learn: 0.4312835	total: 11.6ms	remaining: 71.3ms
14:	learn: 0.4258857	total: 12.4ms	remaining: 70.4ms
15:	learn: 0.4191189	total: 13.2ms	remaining: 69.2ms
16:	learn: 0.4157029	total: 14ms	remaining: 68.3ms
17:	learn: 0.4106709	total: 14.8ms	remaining: 67.4ms
18:	learn: 0.4069488	total: 15.6ms	remaining: 66.7ms
19:	lear

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())


24:	learn: 0.3932717	total: 20ms	remaining: 60.1ms
25:	learn: 0.3916236	total: 20.8ms	remaining: 59.1ms
26:	learn: 0.3896862	total: 21.5ms	remaining: 58.2ms
27:	learn: 0.3876057	total: 22.3ms	remaining: 57.4ms
28:	learn: 0.3854368	total: 23.1ms	remaining: 56.6ms
29:	learn: 0.3838894	total: 23.9ms	remaining: 55.7ms
30:	learn: 0.3823975	total: 24.7ms	remaining: 54.9ms
31:	learn: 0.3807000	total: 25.5ms	remaining: 54.2ms
32:	learn: 0.3794693	total: 26.3ms	remaining: 53.3ms
33:	learn: 0.3779279	total: 27.1ms	remaining: 52.6ms
34:	learn: 0.3764627	total: 27.9ms	remaining: 51.9ms
35:	learn: 0.3749861	total: 28.8ms	remaining: 51.1ms
36:	learn: 0.3736661	total: 29.6ms	remaining: 50.3ms
37:	learn: 0.3723536	total: 30.3ms	remaining: 49.5ms
38:	learn: 0.3703241	total: 31.1ms	remaining: 48.7ms
39:	learn: 0.3687490	total: 31.9ms	remaining: 47.8ms
40:	learn: 0.3679698	total: 32.8ms	remaining: 47.2ms
41:	learn: 0.3667706	total: 33.6ms	remaining: 46.4ms
42:	learn: 0.3659412	total: 34.4ms	remaining: 45

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\2766920084.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model1.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\2766920084.py:23: FutureWarning: Series.ravel is depre

## Label: LONG_INTERACTION

In [19]:
# Label distribution before balancing
print(encoded_data['LONG_INTERACTION_interruptibility'].value_counts())

LONG_INTERACTION_interruptibility
1    1443
0    1387
Name: count, dtype: int64


### Model building and LOSO CV

In [20]:
# Prepare features (X) and target variable (y) for predicting LONG_INTERACTION
X = encoded_data.drop(columns=['LONG_INTERACTION_interruptibility', 'SHORT_INTERACTION_interruptibility','uid'], axis=1)
y = encoded_data['LONG_INTERACTION_interruptibility']
groups = encoded_data['uid']

# Initialize Leave-One-Group-Out cross-validator
logo = LeaveOneGroupOut()

# Initialize SMOTE for oversampling to handle class imbalance
smote = SMOTE(random_state=42)

# Define models to evaluate
models2 = {
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'LightGBM': LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42),
    'CatBoost': CatBoostClassifier(iterations=100, depth=5, learning_rate=0.1, loss_function='Logloss', cat_features=[0], random_seed=42),
    'SVM': SVC(random_state=42),
    'Dummy': DummyClassifier(strategy="stratified", random_state=42)
}

results2_logo = {}

# Loop over each model
for model_name, model2 in models2.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in Leave-One-Group-Out cross-validation
    for i, (train_index, test_index) in enumerate(logo.split(X, y, groups)):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model2.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model2.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)

    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results2_logo[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is depre

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1400, number of negative: 1400
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000240 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 86
[LightGBM] [Info] Number of data points in the train set: 2800, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is depre

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1340, number of negative: 1340
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000252 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 88
[LightGBM] [Info] Number of data points in the train set: 2680, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


...

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


79:	learn: 0.5032813	total: 84.8ms	remaining: 21.2ms
80:	learn: 0.5028117	total: 86.1ms	remaining: 20.2ms
81:	learn: 0.5021335	total: 87.2ms	remaining: 19.1ms
82:	learn: 0.5015836	total: 88.3ms	remaining: 18.1ms
83:	learn: 0.5013004	total: 89.4ms	remaining: 17ms
84:	learn: 0.5010841	total: 90.4ms	remaining: 16ms
85:	learn: 0.5008312	total: 91.4ms	remaining: 14.9ms
86:	learn: 0.5003544	total: 92.4ms	remaining: 13.8ms
87:	learn: 0.4999723	total: 93.4ms	remaining: 12.7ms
88:	learn: 0.4991539	total: 94.4ms	remaining: 11.7ms
89:	learn: 0.4986172	total: 95.6ms	remaining: 10.6ms
90:	learn: 0.4980368	total: 96.7ms	remaining: 9.56ms
91:	learn: 0.4975104	total: 97.7ms	remaining: 8.49ms
92:	learn: 0.4969565	total: 98.6ms	remaining: 7.42ms
93:	learn: 0.4965132	total: 99.6ms	remaining: 6.36ms
94:	learn: 0.4960033	total: 101ms	remaining: 5.3ms
95:	learn: 0.4954534	total: 102ms	remaining: 4.24ms
96:	learn: 0.4951154	total: 103ms	remaining: 3.19ms
97:	learn: 0.4946922	total: 104ms	remaining: 2.12ms
98

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6712263	total: 1.57ms	remaining: 156ms
1:	learn: 0.6540370	total: 2.83ms	remaining: 139ms
2:	learn: 0.6412677	total: 3.77ms	remaining: 122ms
3:	learn: 0.6318057	total: 4.58ms	remaining: 110ms
4:	learn: 0.6212531	total: 5.45ms	remaining: 104ms
5:	learn: 0.6122526	total: 6.63ms	remaining: 104ms
6:	learn: 0.6034983	total: 7.58ms	remaining: 101ms
7:	learn: 0.5956829	total: 8.54ms	remaining: 98.2ms
8:	learn: 0.5896781	total: 9.49ms	remaining: 96ms
9:	learn: 0.5847986	total: 10.4ms	remaining: 93.3ms
10:	learn: 0.5797798	total: 11.3ms	remaining: 91.5ms
11:	learn: 0.5755800	total: 12.2ms	remaining: 89.7ms
12:	learn: 0.5708757	total: 13.1ms	remaining: 87.6ms
13:	learn: 0.5678643	total: 14.3ms	remaining: 87.9ms
14:	learn: 0.5654359	total: 15.4ms	remaining: 87.3ms
15:	learn: 0.5615104	total: 16.6ms	remaining: 87.2ms
16:	learn: 0.5586611	total: 17.5ms	remaining: 85.2ms
17:	learn: 0.5567203	total: 18.3ms	remaining: 83.6ms
18:	learn: 0.5542684	total: 19.2ms	remaining: 81.9ms
19:	learn: 0

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\3640004364.py:40: FutureWarning: Series.ravel is depre

### K-Fold CV

In [21]:
from sklearn.model_selection import KFold

# Initialize 5-fold cross-validator
kfold = KFold(n_splits=5, shuffle=False)

results2_kfold = {}

# Loop over each model
for model_name, model2 in models2.items():
    accuracies = []
    f1_scores = []

    # Loop over each fold in 5-fold cross-validation
    for train_index, test_index in kfold.split(X, y):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply SMOTE to the training data to balance class distribution
        X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

        # Train the model on the oversampled training data
        model2.fit(X_train_oversampled, y_train_oversampled.ravel())

        # Predict the target on the test data
        y_pred = model2.predict(X_test)

        # Evaluate the prediction
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')

        accuracies.append(accuracy)
        f1_scores.append(f1)

    # Compute average accuracy and F1-score across all folds
    avg_accuracy = sum(accuracies) / len(accuracies)
    avg_f1_score = sum(f1_scores) / len(f1_scores)

    results2_kfold[model_name] = {
        'Average Accuracy': avg_accuracy,
        'Average F1-Score (macro)': avg_f1_score
    }

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\1454448026.py:23: FutureWarning: Series.ravel is depre

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1144, number of negative: 1144
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000187 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 86
[LightGBM] [Info] Number of data points in the train set: 2288, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1159, number of negative: 1159
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000253 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 87
[LightGBM] [Info] Number of data points in the train set: 2318, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


85:	learn: 0.4770393	total: 78.9ms	remaining: 12.8ms
86:	learn: 0.4766165	total: 80ms	remaining: 12ms
87:	learn: 0.4762308	total: 81ms	remaining: 11ms
88:	learn: 0.4756301	total: 82.3ms	remaining: 10.2ms
89:	learn: 0.4748644	total: 83.3ms	remaining: 9.25ms
90:	learn: 0.4742615	total: 84.2ms	remaining: 8.32ms
91:	learn: 0.4738147	total: 85.2ms	remaining: 7.41ms
92:	learn: 0.4734412	total: 86.1ms	remaining: 6.48ms
93:	learn: 0.4731078	total: 87ms	remaining: 5.55ms
94:	learn: 0.4718148	total: 87.9ms	remaining: 4.62ms
95:	learn: 0.4713760	total: 88.7ms	remaining: 3.7ms
96:	learn: 0.4707598	total: 89.6ms	remaining: 2.77ms
97:	learn: 0.4701679	total: 90.5ms	remaining: 1.85ms
98:	learn: 0.4696999	total: 91.4ms	remaining: 923us
99:	learn: 0.4689998	total: 92.5ms	remaining: 0us
0:	learn: 0.6715613	total: 1.23ms	remaining: 121ms
1:	learn: 0.6516646	total: 2.38ms	remaining: 117ms
2:	learn: 0.6364084	total: 3.45ms	remaining: 112ms
3:	learn: 0.6247511	total: 4.49ms	remaining: 108ms
4:	learn: 0.6124

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())


0:	learn: 0.6725680	total: 1.27ms	remaining: 126ms
1:	learn: 0.6578199	total: 2.34ms	remaining: 115ms
2:	learn: 0.6400877	total: 19.4ms	remaining: 626ms
3:	learn: 0.6272510	total: 20.5ms	remaining: 491ms
4:	learn: 0.6203358	total: 21.2ms	remaining: 402ms
5:	learn: 0.6112040	total: 22.5ms	remaining: 352ms
6:	learn: 0.6018572	total: 23.5ms	remaining: 312ms
7:	learn: 0.5940544	total: 24.6ms	remaining: 282ms
8:	learn: 0.5876107	total: 25.7ms	remaining: 260ms
9:	learn: 0.5798972	total: 26.8ms	remaining: 241ms
10:	learn: 0.5735825	total: 27.8ms	remaining: 225ms
11:	learn: 0.5684253	total: 29ms	remaining: 212ms
12:	learn: 0.5635740	total: 30ms	remaining: 201ms
13:	learn: 0.5576682	total: 30.8ms	remaining: 189ms
14:	learn: 0.5531824	total: 31.8ms	remaining: 180ms
15:	learn: 0.5498463	total: 32.8ms	remaining: 172ms
16:	learn: 0.5455603	total: 33.8ms	remaining: 165ms
17:	learn: 0.5424235	total: 34.8ms	remaining: 158ms
18:	learn: 0.5395239	total: 35.6ms	remaining: 152ms
19:	learn: 0.5366744	total

C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\1454448026.py:23: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  model2.fit(X_train_oversampled, y_train_oversampled.ravel())
C:\Users\Golibjon\AppData\Local\Temp\ipykernel_6460\1454448026.py:23: FutureWarning: Series.ravel is depre

In [23]:
from tabulate import tabulate

# Models list
models = [
    'Random Forest', 'Gradient Boosting', 'XGBoost',
    'LightGBM', 'CatBoost', 'SVM', 'Dummy'
]

# Prepare table data
table_data = []
for model in models:
    row = [model]
    
    # Shell 1: SHORT_INTERACTION, Leave-One-Group-Out
    row.append(results1_logo.get(model, {}).get('Average Accuracy', 0.0))
    row.append(results1_logo.get(model, {}).get('Average F1-Score (macro)', 0.0))
    
    # Shell 3: LONG_INTERACTION, Leave-One-Group-Out
    row.append(results2_logo.get(model, {}).get('Average Accuracy', 0.0))
    row.append(results2_logo.get(model, {}).get('Average F1-Score (macro)', 0.0))

    # Shell 2: SHORT_INTERACTION, 5-fold
    row.append(results1_kfold.get(model, {}).get('Average Accuracy', 0.0))
    row.append(results1_kfold.get(model, {}).get('Average F1-Score (macro)', 0.0))
    
    # Shell 4: LONG_INTERACTION, 5-fold
    row.append(results2_kfold.get(model, {}).get('Average Accuracy', 0.0))
    row.append(results2_kfold.get(model, {}).get('Average F1-Score (macro)', 0.0))
    
    table_data.append(row)

# Define headers
headers = [
    'Model',
    'LOSO CV\nShort\nInteraction\nAccuracy', 'LOSO CV\nShort\nInteraction\nF1',
    'LOSO CV\nLong\nInteraction\nAccuracy', 'LOSO CV\nLong\nInteraction\nF1',
    '5-fold CV\nShort\nInteraction\nAccuracy', '5-fold CV\nShort\nInteraction\nF1',
    '5-fold CV\nLong\nInteraction\nAccuracy', '5-fold CV\nLong\nInteraction\nF1'
]

# Print the table
print("Machine learning model performance")
print(tabulate(table_data, headers=headers, tablefmt='fancy_grid', floatfmt='.3f', numalign="decimal"))


Machine learning model performance
╒═══════════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╤═══════════════╕
│ Model             │       LOSO CV │       LOSO CV │       LOSO CV │       LOSO CV │     5-fold CV │     5-fold CV │     5-fold CV │     5-fold CV │
│                   │         Short │         Short │          Long │          Long │         Short │         Short │          Long │          Long │
│                   │   Interaction │   Interaction │   Interaction │   Interaction │   Interaction │   Interaction │   Interaction │   Interaction │
│                   │      Accuracy │            F1 │      Accuracy │            F1 │      Accuracy │            F1 │      Accuracy │            F1 │
╞═══════════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╡
│ Random Forest     │         0.828 │         0.746 │         0.6